In [23]:
%matplotlib inline

import numpy as np
import pandas as pd
import os
from sklearn.exceptions import ConvergenceWarning
from sklearn import model_selection, preprocessing, metrics
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import ElasticNetCV, LassoLarsCV, OrthogonalMatchingPursuitCV
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.pipeline import make_pipeline, make_union, Pipeline, FeatureUnion
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import randint
import time
import xgboost as xgb
import csv
import pickle
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [24]:
RAW_DATA_PATH = '/kaggle/dev/mercedes-benz-greener-manufacturing-data/raw_data'
DATA_PATH = '/kaggle/dev/mercedes-benz-greener-manufacturing-data'
TRAIN_DATA = os.path.join(RAW_DATA_PATH, 'train.csv')
TEST_DATA = os.path.join(RAW_DATA_PATH, 'test.csv')
SAMPLE_SUBMISSION = os.path.join(RAW_DATA_PATH, 'sample_submission.csv')
SUBMISSION_PATH = os.path.join(DATA_PATH, 'submissions')
MODELS_PATH = os.path.join(DATA_PATH, 'models')

In [25]:
train_df = pd.read_csv(TRAIN_DATA)
test_df = pd.read_csv(TEST_DATA)
sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION)

In [26]:
filter_outliers = True
xgb_use_augment_features = False
stacked_use_augment_features = True

# Preprocess data
for column in train_df.columns:
    cardinality = len(np.unique(train_df[column]))
    if cardinality == 1:
        train_df.drop(column, axis=1, inplace=True)
        test_df.drop(column, axis=1, inplace=True)

original_train_columns = sorted(list(set(train_df.columns) - set(['ID', 'y'])))
        
for f in ["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]:
    lbl = preprocessing.LabelEncoder()
    train_values = set(train_df[f].values)
    test_values = set(test_df[f].values)
    all_values = list(train_values | test_values)
    lbl.fit(all_values) 
    train_df[f] = lbl.transform(list(train_df[f].values))
    test_df[f] = lbl.transform(list(test_df[f].values))

if filter_outliers:
    print('Filtering outliers')
    # Filter out outlier y = 265.32
    train_df = train_df[train_df.y < 200]

print('train_df.shape', train_df.shape)
print('test_df.shape', test_df.shape)
n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train_df.drop(["y"], axis=1))
tsvd_results_test = tsvd.transform(test_df)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train_df.drop(["y"], axis=1))
pca2_results_test = pca.transform(test_df)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train_df.drop(["y"], axis=1))
ica2_results_test = ica.transform(test_df)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train_df.drop(["y"], axis=1))
grp_results_test = grp.transform(test_df)

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train_df.drop(["y"], axis=1))
srp_results_test = srp.transform(test_df)

# Append decomposition components to datasets
for i in range(0, n_comp):
    train_df['pca_' + str(i)] = pca2_results_train[:, i]
    test_df['pca_' + str(i)] = pca2_results_test[:, i]

    train_df['ica_' + str(i)] = ica2_results_train[:, i]
    test_df['ica_' + str(i)] = ica2_results_test[:, i]

    train_df['tsvd_' + str(i)] = tsvd_results_train[:, i]
    test_df['tsvd_' + str(i)] = tsvd_results_test[:, i]

    train_df['grp_' + str(i)] = grp_results_train[:, i]
    test_df['grp_' + str(i)] = grp_results_test[:, i]

    train_df['srp_' + str(i)] = srp_results_train[:, i]
    test_df['srp_' + str(i)] = srp_results_test[:, i]

augmented_train_columns = sorted(list(set(train_df.columns) - set(['ID', 'y'])))
print('original columns', len(original_train_columns))
print('augmented columns', len(augmented_train_columns))
print('train_df.shape', train_df.shape)
print('test_df.shape', test_df.shape)

Filtering outliers
train_df.shape (4208, 366)
test_df.shape (4209, 365)
original columns 364
augmented columns 424
train_df.shape (4208, 426)
test_df.shape (4209, 425)


In [27]:
X = train_df.drop(["ID", "y"], axis=1)
Y = train_df['y'].values
train_X, holdout_X, train_Y, holdout_Y = model_selection.train_test_split(X, Y, test_size=0.2, random_state=29)
print('X.shape', X.shape)
print('Y.shape', Y.shape)
print('train_X.shape', train_X.shape)
print('train_Y.shape', train_Y.shape)
print('holdout_X.shape', holdout_X.shape)
print('holdout_Y.shape', holdout_Y.shape)

X.shape (4208, 424)
Y.shape (4208,)
train_X.shape (3366, 424)
train_Y.shape (3366,)
holdout_X.shape (842, 424)
holdout_Y.shape (842,)


In [29]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
#         if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
#             X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))
        return X_transformed

NUM_FOLDS = 10
kf = model_selection.KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
perf = []

if stacked_use_augment_features:
    train_X_stacked = train_X[augmented_train_columns]
    holdout_X_stacked = holdout_X[augmented_train_columns]
    test_X_stacked = test_df[augmented_train_columns]
else:
    train_X_stacked = train_X[original_train_columns]
    holdout_X_stacked = holdout_X[original_train_columns]
    test_X_stacked = test_df[original_train_columns]

print('train_X_stacked.shape', train_X_stacked.shape)
print('train_Y.shape', train_Y.shape)
print('holdout_X_stacked.shape', holdout_X_stacked.shape)
print('holdout_Y.shape', holdout_Y.shape)

for fold, (train_idxs, val_idxs) in enumerate(kf.split(train_X_stacked)):
    model_id = "model-" + str(int(time.time()))
    print('Fold {} Training {}'.format(fold, model_id))
    
    trn_X_stacked, val_X_stacked = train_X_stacked.iloc[train_idxs], train_X_stacked.iloc[val_idxs]
    trn_Y_stacked, val_Y_stacked = train_Y[train_idxs], train_Y[val_idxs]
    
    stacked_pipeline = make_pipeline(
        StackingEstimator(estimator=LassoLarsCV(normalize=True)),
        StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber",
                                                              n_estimators=500, max_depth=3,
                                                              max_features=0.55, min_samples_leaf=18,
                                                              min_samples_split=14, subsample=0.7)),
#         ExtraTreesRegressor(n_estimators=500, max_features=0.55,
#                             max_depth=3, min_samples_leaf=18,
#                             min_samples_split=14, n_jobs=-1),
        LassoLarsCV())
    stacked_pipeline.fit(trn_X_stacked, trn_Y_stacked)
    pickle.dump(stacked_pipeline, open(os.path.join(MODELS_PATH, model_id + ".stacked"), "wb"))
    train_Y_stacked_pred = stacked_pipeline.predict(trn_X_stacked)
    train_r2_score = metrics.r2_score(trn_Y_stacked, train_Y_stacked_pred)
    val_Y_stacked_pred = stacked_pipeline.predict(val_X_stacked)
    val_r2_score = metrics.r2_score(val_Y_stacked, val_Y_stacked_pred)
    holdout_Y_pred = stacked_pipeline.predict(holdout_X_stacked)
    holdout_r2_score = metrics.r2_score(holdout_Y, holdout_Y_pred)
    perf.append((fold, model_id, train_r2_score, val_r2_score, holdout_r2_score))

perf_stacked_df = pd.DataFrame(perf, columns=['fold', 'model_id', 'train_r2', 'validation_r2', 'holdout_r2'])
print('avg train r2', perf_stacked_df['train_r2'].mean(),
      'avg validation r2', perf_stacked_df['validation_r2'].mean(), 
      'avg holdout r2', perf_stacked_df['holdout_r2'].mean())
perf_stacked_df

train_X_stacked.shape (3366, 424)
train_Y.shape (3366,)
holdout_X_stacked.shape (842, 424)
holdout_Y.shape (842,)
Fold 0 Training model-1498061991


/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))
/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))


Fold 1 Training model-1498062003


/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))
/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))


Fold 2 Training model-1498062014


...

/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))


Fold 9 Training model-1498062093


/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))


avg train r2 0.612225491464 avg validation r2 0.599630696363 avg holdout r2 0.589464914707


,fold,model_id,train_r2,validation_r2,holdout_r2
0,0,model-1498061991,0.604479,0.681476,0.587035
1,1,model-1498062003,0.619481,0.541933,0.590708
2,2,model-1498062014,0.606163,0.658690,0.589458
3,3,model-1498062025,0.603385,0.663311,0.590640
4,4,model-1498062037,0.609703,0.619665,0.590944
5,5,model-1498062048,0.618877,0.500446,0.586479
6,6,model-1498062060,0.620761,0.537532,0.593470
7,7,model-1498062071,0.622527,0.520586,0.587939
8,8,model-1498062082,0.603501,0.680335,0.587390
9,9,model-1498062093,0.613380,0.592334,0.590586


In [20]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))
        return X_transformed

if stacked_use_augment_features:
    train_X_stacked = train_X[augmented_train_columns]
    holdout_X_stacked = holdout_X[augmented_train_columns]
    test_X_stacked = test_df[augmented_train_columns]
else:
    train_X_stacked = train_X[original_train_columns]
    holdout_X_stacked = holdout_X[original_train_columns]
    test_X_stacked = test_df[original_train_columns]

print('train_X_stacked.shape', train_X_stacked.shape)
print('train_Y.shape', train_Y.shape)
print('holdout_X_stacked.shape', holdout_X_stacked.shape)
print('holdout_Y.shape', holdout_Y.shape)

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber",
                                                          n_estimators=500, max_depth=3,
                                                          max_features=0.55, min_samples_leaf=18,
                                                          min_samples_split=14, subsample=0.7)),
    LassoLarsCV())
stacked_pipeline.fit(train_X_stacked, train_Y)
train_Y_stacked_pred = stacked_pipeline.predict(train_X_stacked)
train_r2_score = metrics.r2_score(train_Y, train_Y_stacked_pred)
holdout_Y_stacked_pred = stacked_pipeline.predict(holdout_X_stacked)
holdout_r2_score = metrics.r2_score(holdout_Y, holdout_Y_stacked_pred)

print('train R2', train_r2_score, 'holdout R2', holdout_r2_score)

train_X_stacked.shape (3366, 424)
train_Y.shape (3366,)
holdout_X_stacked.shape (842, 424)
holdout_Y.shape (842,)


/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))


train R2 0.613272076897 holdout R2 0.591621963838


In [8]:
NUM_FOLDS = 10
kf = model_selection.KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
perf = []

train_X_original_columns = train_X[original_train_columns]
holdout_X_original_columns = holdout_X[original_train_columns]

if xgb_use_augment_features:
    train_X_xgb = train_X[augmented_train_columns]
    test_X_xgb = test_df[augmented_train_columns]
    holdout_X_xgb = holdout_X[augmented_train_columns]
else:
    train_X_xgb = train_X[original_train_columns]
    test_X_xgb = test_df[original_train_columns]
    holdout_X_xgb = holdout_X[original_train_columns]

print('train_X_xgb.shape', train_X_xgb.shape)
print('train_Y.shape', train_Y.shape)
print('holdout_X_xgb.shape', holdout_X_xgb.shape)
print('holdout_Y.shape', holdout_Y.shape)
print('test_X_xgb.shape', test_X_xgb.shape, '\n')

for fold, (train_idxs, val_idxs) in enumerate(kf.split(train_X_xgb)):
    model_id = "model-" + str(int(time.time()))
    print('Fold {} Training {}'.format(fold, model_id))
    
    trn_X, val_X = train_X_xgb.iloc[train_idxs], train_X_xgb.iloc[val_idxs]
    trn_Y, val_Y = train_Y[train_idxs], train_Y[val_idxs]

    def xgb_r2_score(preds, dtrain):
        labels = dtrain.get_label()
        return 'rmse', -1.0 * metrics.r2_score(labels, preds)

    model = xgb.XGBRegressor(max_depth = 10,
                        gamma=0.5,
                        objective="reg:linear",
                        n_estimators=1000,
                        learning_rate=0.005,
                        nthread=12,
                        subsample=0.8,
                        colsample_bytree=0.70,
                        colsample_bylevel=0.70,
                        #base_score=Y_mean,
                        seed=42,
                        silent=True)

    model.fit(trn_X, trn_Y, eval_set=[(trn_X, trn_Y), (val_X, val_Y)], verbose=False, eval_metric='rmse', early_stopping_rounds=50)
    evals_result = model.evals_result()
    pickle.dump(model, open(os.path.join(MODELS_PATH, model_id + ".xgb"), "wb"))
    train_Y_pred = model.predict(trn_X)
    train_r2_score = metrics.r2_score(trn_Y, train_Y_pred)
    val_Y_pred = model.predict(val_X)
    val_r2_score = metrics.r2_score(val_Y, val_Y_pred)
    holdout_Y_pred = model.predict(holdout_X_xgb)
    holdout_r2_score = metrics.r2_score(holdout_Y, holdout_Y_pred)
    perf.append((fold, model_id, train_r2_score, val_r2_score, holdout_r2_score))

perf_df = pd.DataFrame(perf, columns=['fold', 'model_id', 'train_r2', 'validation_r2', 'holdout_r2'])
print('avg train r2', perf_df['train_r2'].mean(),
      'avg validation r2', perf_df['validation_r2'].mean(), 
      'avg holdout r2', perf_df['holdout_r2'].mean())
perf_df

train_X_xgb.shape (3366, 364)
train_Y.shape (3366,)
holdout_X_xgb.shape (842, 364)
holdout_Y.shape (842,)
test_X_xgb.shape (4209, 364) 

Fold 0 Training model-1498059343
Fold 1 Training model-1498059347
Fold 2 Training model-1498059350
Fold 3 Training model-1498059353
Fold 4 Training model-1498059357
Fold 5 Training model-1498059360
Fold 6 Training model-1498059364
Fold 7 Training model-1498059367
Fold 8 Training model-1498059370
Fold 9 Training model-1498059374
avg train r2 0.782674286221 avg validation r2 0.574559045576 avg holdout r2 0.568158969746


,fold,model_id,train_r2,validation_r2,holdout_r2
0,0,model-1498059343,0.778743,0.666772,0.571399
1,1,model-1498059347,0.782076,0.516920,0.566121
2,2,model-1498059350,0.781809,0.627925,0.572334
3,3,model-1498059353,0.755803,0.669401,0.569425
4,4,model-1498059357,0.780663,0.594206,0.569809
5,5,model-1498059360,0.808240,0.452921,0.560852
6,6,model-1498059364,0.783474,0.526206,0.569820
7,7,model-1498059367,0.795202,0.478438,0.565369
8,8,model-1498059370,0.785030,0.641142,0.561955
9,9,model-1498059374,0.775703,0.571660,0.574505


In [9]:
# Avg xgb models predictions
VALIDATION_THRESHOLD = 0.4
NUM_MODELS = perf_df[perf_df.validation_r2 > VALIDATION_THRESHOLD].shape[0]
holdout_Y_pred = np.zeros(shape=(holdout_X.shape[0], NUM_MODELS), dtype=np.float32)
test_Y_pred = np.zeros(shape=(test_df.shape[0], NUM_MODELS), dtype=np.float32)

count = 0
for row in perf_df.itertuples():
    fold, model_id, train_r2, validation_r2 = row[1], row[2], row[3], row[4]
    if validation_r2 < VALIDATION_THRESHOLD:
        continue
    model = pickle.load(open(os.path.join(MODELS_PATH, model_id + '.xgb'), 'rb'))
    test_Y_pred[:, count] = model.predict(test_X_xgb)
    holdout_Y_pred[:, count] = model.predict(holdout_X_xgb)
    count += 1

holdout_Y_avg = np.mean(holdout_Y_pred, axis=1)
test_Y_avg = np.mean(test_Y_pred, axis=1)
test_Y_avg.shape

(4209,)

In [229]:
# Avg stacked models predictions
VALIDATION_THRESHOLD = 0.4
NUM_MODELS = perf_stacked_df[perf_stacked_df.validation_r2 > VALIDATION_THRESHOLD].shape[0]
holdout_Y_pred = np.zeros(shape=(holdout_X.shape[0], NUM_MODELS), dtype=np.float32)
test_Y_pred = np.zeros(shape=(test_df.shape[0], NUM_MODELS), dtype=np.float32)

count = 0
for row in perf_stacked_df.itertuples():
    fold, model_id, train_r2, validation_r2 = row[1], row[2], row[3], row[4]
    if validation_r2 < 0.4:
        continue
    model = pickle.load(open(os.path.join(MODELS_PATH, model_id + '.stacked'), 'rb'))
    test_Y_pred[:, count] = model.predict(test_X_stacked)
    holdout_Y_pred[:, count] = model.predict(holdout_X_stacked)
    count += 1

holdout_Y_avg_stacked = np.mean(holdout_Y_pred, axis=1)
test_Y_avg_stacked = np.mean(test_Y_pred, axis=1)
test_Y_avg_stacked.shape

(4209,)

In [214]:
# Weighted average of predictions
holdout_Y_ensembled = holdout_Y_avg * 0.5 + holdout_Y_avg_stacked * 0.5
holdout_r2_ensemble = metrics.r2_score(holdout_Y, holdout_Y_ensembled)
print('holdout_r2_ensemble', holdout_r2_ensemble)

test_Y_ensemble_1 = test_Y_avg * 0.5 + test_Y_avg_stacked * 0.5
test_Y_ensemble_1.shape

holdout_r2_ensemble 0.583066122756


(4209,)

In [16]:
# Weighted average of predictions
# holdout_Y_ensembled = holdout_Y_avg * 0.5 + holdout_Y_stacked_pred * 0.5
# holdout_r2_ensemble = metrics.r2_score(holdout_Y, holdout_Y_ensembled)
# print('holdout_r2_ensemble', holdout_r2_ensemble)

# Single stacked model prediction
test_X_xgb = test_df[augmented_train_columns]
stacked_Y = stacked_pipeline.predict(test_X_stacked)
# stacked_Y.shape
test_Y_ensemble_2 = test_Y_avg * 0.5 + stacked_Y * 0.5
test_Y_ensemble_2.shape

(4209,)

In [17]:
# Create submission file
submission_df = test_df[['ID']]
submission_df['y'] = test_Y_ensemble_2.tolist()
submission_df.to_csv(os.path.join(SUBMISSION_PATH, 'submission-' + str(int(time.time())) + '.csv'), index=False)
print('Generated submission ', os.path.join(SUBMISSION_PATH, 'submission-' + str(int(time.time())) + '.csv'))

Generated submission  /kaggle/dev/mercedes-benz-greener-manufacturing-data/submissions/submission-1498059783.csv


/kaggle/dev/ashish/mercedes-benz-greener-manufacturing/merc/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
